In [18]:
#sample_rate, preemphasis, min_db, max_val, zero, num_pitch, min_pitch, num_phone, consonant

#22050, 0.97, -80.0, 1.0, 'mean', 29, 51, korean_g2p.num_phone, 3

In [1]:
import os
import numpy as np
import librosa
import madmom.utils.midi as midi
from scipy import signal
from tsmpy.wsolatsm import wsola

/Users/seungheondoh/anaconda3/envs/python3.7/lib/python3.7/site-packages/madmom/utils/midi.py:116: UserWarning: Deprecated as of version 0.16. Please use madmom.io.midi instead. This module will be removed in version 0.18.
  warnings.warn('Deprecated as of version 0.16. Please use madmom.io.midi '


In [4]:
def wsola_pitch_shift(x, sr, pitch_shift):
    beta = 2**(pitch_shift/12.0)
    x_resample = librosa.resample(x, sr*beta, sr)
    x_wsola = wsola(x_resample, beta)
    return x_wsola

In [10]:
filename = "input/pop.wav"
y, sr = librosa.load(filename, 22050)
print(y.shape)

(661504,)


In [11]:
import IPython.display as ipd
ipd.Audio(y, rate=sr)

In [12]:
y = wsola_pitch_shift(y, sr, 1)
print(y.shape)
ipd.Audio(y, rate=sr)

(661505,)


In [14]:
# https://dsp.stackexchange.com/questions/19084/applying-filter-in-scipy-signal-use-lfilter-or-filtfilt
def preemphasis(x, preemphasis = 0.97):
    return signal.lfilter([1,-preemphasis],[1],x)

In [15]:
preemphasis(y)

array([-0.04585416,  0.01148532,  0.01224996, ...,  0.        ,
        0.        ,  0.        ])

stft

In [24]:
spec = librosa.stft(y=y, n_fft=512, hop_length=128)
print(spec.shape)
spec[0][1000]

(257, 5169)


(-0.2162592+0j)

move to db

In [19]:
def amp_normalize(S, min_db=-80.0,max_val=1.0, zero='mean'):
    if zero is "min":
        S_norm = (S-min_db)/(-min_db)
        S_norm = np.clip(S_norm, 0, max_val)
    elif zero is "mean":
        S_norm = (S-min_db/2.0)/(-min_db/2.0)
        S_norm = np.clip(S_norm, -max_val, max_val)
    return S_norm

In [25]:
spec_db = librosa.amplitude_to_db(np.abs(spec), amin=1e-5, top_db=None)
spec_db[0][1000]

-13.3005085

In [26]:
spec_db = amp_normalize(spec_db)
spec_db[0][1000]

0.66748726

mag-phase

In [30]:
y, sr = librosa.load(filename, sr=22050)
spec = librosa.stft(y=y, n_fft=512, hop_length=128)
mag, phs = librosa.magphase(spec)
mag.shape, phs.shape

((257, 5169), (257, 5169))

In [33]:
output = np.zeros((2,spec.shape[1], spec.shape[0]))
output.shape

(2, 5169, 257)

In [35]:
mag_db = librosa.amplitude_to_db(mag)
mag_db = amp_normalize(mag_db)
output[0] = mag_db.T

In [36]:
def phase_normalize(phase):
    return phase/np.pi

In [44]:
phs[0][0:5]

array([-1.+1.509958e-07j, -1.+1.509958e-07j,  1.+0.000000e+00j,
        1.+0.000000e+00j, -1.+1.509958e-07j], dtype=complex64)

In [45]:
phs_unwarp = np.unwrap(np.angle(phs))
phs_unwarp[0][0:5]

array([3.1415925, 3.1415925, 0.       , 0.       , 3.1415925])

In [47]:
inst_freq = np.zeros(phs_unwarp.shape)
inst_freq[:,1:] = phs_unwarp[:,1:] - phs_unwarp[:,:-1]
inst_freq[0][0:5]

array([ 0.       ,  0.       , -3.1415925,  0.       ,  3.1415925])

In [48]:
inst_freq = phase_normalize(inst_freq)
inst_freq[0][0:5]

array([ 0.        ,  0.        , -0.99999995,  0.        ,  0.99999995])

In [49]:
output[1] = inst_freq.T

In [64]:
output[0][0][150:200]

array([ 0.64011955,  0.85059816,  0.80047095,  0.73635328,  0.71568644,
        0.31710833,  0.66144216,  0.54408187,  0.74910975,  0.65131855,
        0.67182124,  0.91514957,  0.95982587,  0.75895041, -0.04484282,
        0.39147419,  0.42606822,  0.47934598,  0.59674376,  0.59228212,
        0.69922245,  0.75449085,  0.821558  ,  1.        ,  0.88666981,
        0.050317  ,  0.78290308,  0.73924053,  0.63469648,  0.77563733,
        0.69321644,  0.2325625 ,  0.79040277,  0.86750668,  0.71832329,
        0.18261461,  0.621512  ,  0.31801528,  0.63221568,  0.70599115,
        0.52965635,  0.36729091,  0.90738946,  0.91684735,  0.91040802,
        1.        ,  1.        ,  0.93650246,  0.84158784,  0.34642845])

In [65]:
output[1][0][150:200]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [70]:
spec_db.shape

(257, 5169)

In [ ]:
array_strided = 0 
array = spec_db[:]